# 参考・引用
[House Prices EDA](https://www.kaggle.com/dgawlik/house-prices-eda)

[HousePrice - data cleaning & visualization](https://www.kaggle.com/code1110/houseprice-data-cleaning-visualization)

# 概要
1. EDAとデータの前処理
  - Profile Reportの作成
  - Label Encode
  - NaN(欠損データ)への対処
  - 新しい列(feature)の作成
  - SalesPriceが正規分布に従っているか
  - ランダムフォレストで特徴量の重要度にアクセス
  - 標準化(Standardization)で特徴量の尺度を揃える
  - 外れ値(outlier)を削除
2. モデルの作成
  - 線形回帰分析での予測

# ライブラリについて
機械学習の実装を行うに当たって、役に立つライブラリについて説明。

## Pandas
データ解析を支援する機能を提供するライブラリ。数表や時系列データを操作するためのデータ構造と演算を提供。

### 特徴
- データ操作のための高速で効率的なDataFrame (行列型) オブジェクト
- メモリ内のデータ構造と複数のフォーマット(CSV, TXT, xls, xlsx, )のデータ間で相互に読み書きするためのツール群
- データ処理(カウント, 集計, 変換, 欠損値処理, マージ, 結合, etc...)
- 時系列データ処理(日, 週, 月, 四半期, 年)

## pandas-profiling
pandasのDataFrameのProfile Reportを作成するライブラリ。

### Profile Reportの内容
- Overview(概要)
  - Number of variables(列数)
  - Number of observations(データ行数)
  - Warning(欠損値やゼロ値の数と割合など)
  - etc...
- Variable(列)
  - Distinct count(登録されたデータの種類の数。登録されたデータがTrueとFalseだけの場合は2)
  - mean(平均)
  - etc...
- Correlations(相関)
- Sample(データの先頭5行)
  

## Matplotlib
グラフ描画の為のライブラリ。

### グラフの種類
- 折れ線グラフ
- 散布図
- ヒストグラム
- ヒートマップ
- 3Dグラフ

## Seaborn
Matplotlibの機能を**より美しく、またより簡単**に実現するためのライブラリ。

### グラフの種類
- 折れ線グラフ
- 散布図
- ヒストグラム
- ヒートマップ

## Numpy
プログラミング言語Pythonにおいて数値計算を効率的に行うためのライブラリ。

効率的な数値計算を行うための型付きの多次元配列（例えばベクトルや行列など）のサポートをPythonに加えるとともに、それらを操作するための大規模な高水準の数学関数ライブラリを提供。

## scikit-learn
Pythonのオープンソース機械学習ライブラリ。
Pythonの数値計算ライブラリのNumPyとSciPyとやり取りするよう設計されている。


In [ ]:
# No.1
# ライブラリのインポート
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression

# Jupyter Notebookの中でインライン表示する場合の設定（これが無いと別ウィンドウでグラフが開く）
%matplotlib inline

# データの読み込み
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")


# 1. EDAとデータの前処理
## EDAとは？
「Explanatory Data Analysis」の略で、日本語だと「探索的データ解析」。

データの特徴を探求し、構造を理解することを目的としたデータサイエンスの最初の一歩の事を指す。

データに触れてみて、データを視覚化したり、データのパターンを探したり、特徴量やターゲットの関係性/相関性を感じるとるのが目的。

## Kaggle上に保存されているデータの確認
画面右上の < ボタンをクリック後、Dataタブをクリックすると、本環境に保存されているデータの一覧が表示される。

## Profile Reportの作成
データの前処理を実施する前に、どのようなデータを扱っているのか、どのような状況にあるのかを、しっかりと理解するのが重要。

トレーニングデータとテストデータのProfile Reportを作成。

In [ ]:
# No.2
# トレーニングデータのProfile Reportを作成
# (出力結果が膨大なのでコメントアウト。必要な時だけ実行)
# pandas_profiling.ProfileReport(train)

In [ ]:
# No.3
# テストデータのProfile Reportを作成
# (出力結果が膨大なのでコメントアウト。必要な時だけ実行)
# pandas_profiling.ProfileReport(test)

# Label Encode
取り扱うデータのデータタイプを確認して、objetctタイプのデータが存在した場合は、Label Encodeする。

## 理由
int64、float64は数値データの為、そのまま扱えることが可能。

しかし、objectは基本的には文字情報で、このままだと扱う事は不可能。(機械学習の実体は数式を使ったプログラムの為)
    
Label Encoderを使ってobjectをラベル化した数値に変換する必要がある。


In [ ]:
# No.4
# データタイプの確認
train.dtypes

In [ ]:
# No.5
# Label Encodeする前の、SaleConditionカラムにセットされている値の一覧
train["SaleCondition"].unique()

In [ ]:
#　No.6
# 文字列をラベル化した数値に変換する為のライブラリをインポート
from sklearn.preprocessing import LabelEncoder

# データタイプがobjectの列の値をラベル化した数値に変換
for i in range(train.shape[1]):
    if train.iloc[:,i].dtypes == object:
        lbl = LabelEncoder()
        lbl.fit(list(train.iloc[:,i].values) + list(test.iloc[:,i].values))
        train.iloc[:,i] = lbl.transform(list(train.iloc[:,i].values))
        test.iloc[:,i] = lbl.transform(list(test.iloc[:,i].values))

In [ ]:
# No.7
# Label Encodeされた後の、SaleConditionカラムにセットされている値の一覧
train["SaleCondition"].unique()

# NaN(欠損値)の対処
多くの場合、データにはNaNが発生している。NaNをそのままにしておくとモデルの精度に影響がある為、何らかの措置が必要。

In [ ]:
# No.8
# トレーニングデータのNaNの数
train_nan = train.isnull().sum()
train_nan = train_nan[train_nan > 0]
train_nan

In [ ]:
# No.9
# テストデータのNaNの数
test_nan = test.isnull().sum()
test_nan = test_nan[test_nan > 0]
test_nan

In [ ]:
# No.10
# keep ID for submission
train_ID = train['Id']
test_ID = test['Id']

# split data for training
y_train = train['SalePrice']
X_train = train.drop(['Id','SalePrice'], axis=1)
X_test = test.drop('Id', axis=1)

# dealing with missing data
Xmat = pd.concat([X_train, X_test])
# 欠損値の多いカラムを削除
Xmat = Xmat.drop(['LotFrontage','MasVnrArea','GarageYrBlt'], axis=1)
# 欠損値の少ないカラムのNaNは中央値(median)で埋める
Xmat = Xmat.fillna(Xmat.median())

# check whether there are still nan
Xmat_nan = Xmat.isnull().sum()
Xmat_nan = Xmat_nan[Xmat_nan > 0]
Xmat_nan

# 新しい列(feature)の作成
良い予測をする為のfeatureを、既存のfeatureを組み合わせて作成

例. 地下室の面積(TotalBsmtSF)と1階のフロアの面積(1stFlrSF)と2階のフロアの面積(2ndFlrSF)を組み合わせて、家全体の面積(TotalSF)を作成

In [ ]:
# No.11
Xmat["TotalSF"] = Xmat["TotalBsmtSF"] + Xmat["1stFlrSF"] + Xmat["2ndFlrSF"]

# SalesPriceが正規分布に従っているか

## 機械学習における正規分布の効果
予測しようとしている値が正規分布に従った方が精度がよくなる

## ヒストグラムで分析
- 横軸にSalesPrice
- 縦軸に割合

In [ ]:
# No.12
sns.distplot(y_train)
plt.show()

正規分布(グラフにしたときに数値の大半が中央に集中し、左右対称の釣り鐘型に「分布」するデータ)ではない

In [ ]:
# No.13
y_train.head()

In [ ]:
# No.14
y_train.sort_values(ascending=False).head()

In [ ]:
# No.15
# 対数計算を実施
# 数字のばらつき、偏りを小さくする
y_train = np.log(y_train)

sns.distplot(y_train)
plt.show()

## 対数計算とは
何乗すればよいかを求める計算

### e.g.
log10(100) = 10を何乗すれば100になるか？ = 2

### 今回の計算
log(y_train) = log np.e(y_train)

※np.e=2.718281828459045(Numpyの定数として定義されている)

log2.718281828459045(755000) = 13.534473

log2.718281828459045(140000) = 11.849398

# ランダムフォレストで特徴量の重要度にアクセス
相関係数でFeatureの重要度を判断するのではなく、機械学習のアルゴリズムであるランダムフォレストでFeatureの重要度を判断。

In [ ]:
# No.16
# trainデータとtestデータを含んでいるXmatを、再度trainデータとtestデータに分割
X_train = Xmat.iloc[:train.shape[0],:]
X_test = Xmat.iloc[train.shape[0]:,:]

# ランダムフォレストをインポート
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=80, max_features='auto')
rf.fit(X_train, y_train)
print("Training done using Random Forest")

# np.argsort()はソート結果の配列のインデックスを返す。引数の頭に"-"をつけると降順。
# つまり"-rf.feature_importances_"を引数にする事で重要度の高い順にソートした配列のインデックスを返す。
ranking = np.argsort(-rf.feature_importances_)
f, ax = plt.subplots(figsize=(11, 9))
sns.barplot(x=rf.feature_importances_[ranking], y=X_train.columns.values[ranking], orient='h')
ax.set_xlabel("feature importance")
plt.tight_layout()
plt.show()

OverallQualとTotalSFの重要度が突出して高いことが判明。

用意されたデータの中でSalesPriceの予測に必要なfeatureは、少ししかない事が判明。

とりあえずは上位30個のFeatureを取ってきて、それをモデルに与え、訓練を実施。

上位2個のFeatureを掛け合わせたものを、'Interaction'として新しいFeatureを作成。

In [ ]:
# No.17
# use the top 30 features only
X_train = X_train.iloc[:,ranking[:30]]
X_test = X_test.iloc[:,ranking[:30]]

# interaction between the top 2
X_train["Interaction"] = X_train["TotalSF"] * X_train["OverallQual"]
X_test["Interaction"] = X_test["TotalSF"] * X_test["OverallQual"]

# 標準化(Standardization)で特徴量の尺度を揃える
特徴量、つまりfeatureが複数ある場合、featureによって数字の大きさや単位が異なる。

例えば、カラムAの値が300cmでカラムBの値が3mの場合、何もしなければ値がそのまま代入されてしまうため、影響度が同じであってもカラムAの影響が大きくなってしまう。

y = 100 + θ1 * 3,000 + θ2 * 10

標準化は、この数字の大きさや単位の影響をなくしてしまい、それぞれの尺度を同じ尺度の値に揃えるための手法。

データを一定の方法で変形し、例えば身長と体重みたいな次元が違うものに対しても、なんとかして同じような単位で取り扱えるようにして、計算や比較しやすくしようというのが狙いのもの。

In [ ]:
# No.18
# z-scoreにて標準化
# (値 - 平均) / 標準偏差
X_train = (X_train - X_train.mean()) / X_train.std()
X_test = (X_test - X_test.mean()) / X_test.std()

## SalePriceと各Featureの関係を調べる
SalePrice / 各Featureのプロット図を表示

In [ ]:
# No.19
# relation to the target
fig = plt.figure(figsize=(12,7))
for i in np.arange(30):
    ax = fig.add_subplot(5,6,i+1)
    sns.regplot(x=X_train.iloc[:,i], y=y_train)

plt.tight_layout()
plt.show()

# 外れ値(outlier)を削除
TotalSFとGrLivAreaの外れ値を削除


In [ ]:
# No.20
# outlier deletion
Xmat = X_train
Xmat['SalePrice'] = y_train
Xmat = Xmat.drop(index = Xmat[(Xmat['TotalSF'] > 5) & (Xmat['SalePrice'] < 12.5)].index)
Xmat = Xmat.drop(index = Xmat[(Xmat['GrLivArea'] > 5) & (Xmat['SalePrice'] < 13)].index)

# recover
y_train = Xmat['SalePrice']
X_train = Xmat.drop(['SalePrice'], axis=1)

以上で、EDAパートは終了。

# モデルの作成
## 線形回帰分析での予測

In [ ]:
# No.21
# アルゴリズムに線形回帰(Linear Regression)を採用
slr = LinearRegression()

# fit関数で学習開始
slr.fit(X_train,y_train)

# 偏回帰係数(回帰分析において得られる回帰方程式の各説明変数の係数)を出力
# 偏回帰係数はscikit-learnのcoefで取得
print('傾き：{0}'.format(slr.coef_[0]))

# y切片(直線とy軸との交点)を出力
# 余談：x切片もあり、それは直線とx軸との交点を指す
print('y切片: {0}'.format(slr.intercept_))

In [ ]:
# No.22
# 学習済みのモデルから予測した結果をセット
# logで小さくなった尺度をexpで戻す
y_test_pred = np.exp(slr.predict(X_test))

In [ ]:
# No.23
# 学習済みのモデルから予測した結果を出力
y_test_pred

## Kaggleにサブミットするファイル形式に変換
作成したモデルをKaggleに提出して評価を受ける為に、[決められた提出フォーマット](https://www.kaggle.com/c/house-prices-advanced-regression-techniques#evaluation)に変換します。
- Id、SalePriceの2列のファイル

In [ ]:
# No.24
# submission
submission = pd.DataFrame({
    "Id": test_ID,
    "SalePrice": y_test_pred
})
submission.to_csv('submission.csv', index=False)

## Kaggleへの提出と評価
### 提出ファイルの作成
画面右上の Commit & Run ボタンをクリック。

そうすることで、Kaggleの所定位置に上記の処理で作成されたcsvデータがセットされる。

### ファイルの提出と評価
画面右上の > ボタンをクリックして、Versions タブをクリックして、Outputタブをクリック。

その中にある、 Submit to Competitionをクリック。